# Extract Articles URLS from ESPN.com

Requirements:

* beautifulsoup4  (4.6.0)

In [2]:
from utils import read_txt
from scrape import get_df_from_teamname_links

In [9]:
df_path = r'data/ESPN_football.csv'
link_path = "data/ESPN_teamname_links.txt"

In [4]:
links = read_txt(link_path)
dfs = get_df_from_teamname_links(links)

dfs.head(20)

Teampage: http://www.espn.com/nfl/team/_/name/buf/buffalo-bills
Teampage: http://www.espn.com/nfl/team/_/name/mia/miami-dolphins
Teampage: http://www.espn.com/nfl/team/_/name/cle/cleveland-browns
Teampage: http://www.espn.com/nfl/team/_/name/ne/new-england-patriots
Teampage: http://www.espn.com/nfl/team/_/name/nyj/new-york-jets
Teampage: http://www.espn.com/nfl/team/_/name/dal/dallas-cowboys
Teampage: http://www.espn.com/nfl/team/_/name/nyg/new-york-giants
Teampage: http://www.espn.com/nfl/team/_/name/phi/philadelphia-eagles
Teampage: http://www.espn.com/nfl/team/_/name/wsh/washington-redskins
Teampage: http://www.espn.com/nfl/team/_/name/bal/baltimore-ravens
Teampage: http://www.espn.com/nfl/team/_/name/cin/cincinnati-bengals
Teampage: http://www.espn.com/nfl/team/_/name/cle/cleveland-browns
Teampage: http://www.espn.com/nfl/team/_/name/pit/pittsburgh-steelers
Teampage: http://www.espn.com/nfl/team/_/name/chi/chicago-bears
Teampage: http://www.espn.com/nfl/team/_/name/det/detroit-lion

,author,class,data-id,sport,teamname,timestamp,url
0,Dan Graziano,story-link,26744212,nfl,buffalo-bills,17h,http://www.espn.com/nfl/story/_/id/26744212/ra...
1,ESPN NFL Experts,story-link,26731919,nfl,buffalo-bills,17h,http://www.espn.com/nfl/story/_/id/26731919/nf...
2,Mike Rodak,story-link,buffalo-bills-33194,nfl,buffalo-bills,3d,http://espn.com/blog/buffalo-bills/post/_/id/3...
3,ESPN NFL Experts,story-link,26679818,nfl,buffalo-bills,6d,http://www.espn.com/nfl/draft2019/story/_/id/2...
4,Hank Gargiulo and Seth Walder,story-link,26696757,nfl,buffalo-bills,7d,http://www.espn.com/nfl/story/_/id/26696757/me...
5,Mel Kiper Jr.,story-link,26678391,nfl,buffalo-bills,10d,http://insider.espn.com/nfl/draft2019/insider/...
6,Mike Rodak,story-link,buffalo-bills-33181,nfl,buffalo-bills,13d,http://espn.com/blog/buffalo-bills/post/_/id/3...
7,Mike Rodak,story-link,26650343,nfl,buffalo-bills,14d,http://www.espn.com/nfl/story/_/id/26650343/so...
8,Joe Klingele,story-link,26632812,nfl,buffalo-bills,17d,http://www.espn.com/chalk/story/_/id/26632812/...
9,Mike Rodak,story-link,buffalo-bills-33171,nfl,buffalo-bills,18d,http://espn.com/blog/buffalo-bills/post/_/id/3...


In [ ]:
# Add a column with just the base url (which represent different high-level types of articles)
df['base_url'] = df.url.apply(lambda x : '/'.join(x.split('/')[:4]))

#### Save dataset with urls and article metadata

In [6]:
export_csv = dfs.to_csv(df_path, index=False) 